# Data Cleaning & Validation Practice

In the following Notebook, I will be cleaning the Welcome Trust APC Spend(2012-2013) dataset utilizing what I have learned about RegEx. This dataset is composed of article open-access prices. From the README File:

**Overview**:

In an attempt to make the debate around the costs of open access publishing more evidence based, the Welcome Trust is releasing into the public domain details of its open access spending in the year 2012-2013, as reported by UK institutions and the Trust’s Major Overseas Programmes in receipt of an OA block grant (see: http://www.wellcome.ac.uk/About-us/Policy/Spotlight-issues/Open-access/Guides/WTX036803.htm).

The data only includes information when an APC was levied. If an author has self-archived a paper, or the publisher provides a gold OA service (but makes no charge), this information is not included in this dataset.

Equally, data  are not included in cases where a researcher (based at an institution not in receipt of an OA block grant) received a supplement to their grant to cover OA publishing costs.

**Assignment**
* Clean Dataset to the best of your ability
* Determine the five most common journals and the total articles for each. 
* Calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

### Read in libraries

In [4]:
import pandas as pd
import re
import numpy as np
import matplotlib as plt

### Read in Data

In [5]:
file = open("Welcome Trust APC spend (2012-13) - Original.csv", errors='ignore')
df = pd.read_csv(file, encoding="utf8")

## About the Data

The dataset contains the following elements:

**PMC ID/PMID**:To help the Trust identify whether a work is in PMC/Europe PMC, we ask institutions to cite the PMC ID (or the PubMed ID if a PMC ID is not known).  This data is not always 100% accurate.

**Publisher**: This lists the name of the publisher, as reported by the institution.  As we do not impose any name authority control on this field, you will see the same publisher listed in different ways.  For example, OUP, Oxford University Press, and O.U.P.

**Journal name**: This lists the name of the journal (which published the Wellcome-funded work), as reported by the institution.  Again, as we not impose any name authority control on this, you will see the same journal listed in different ways.  For example, PNAS, Proceedings of the National Academy of Sciences, P.N.A.S.

**Article title**: This is the title of the article, as reported by the institution.

**Cost**: The cost listed is the figure which the institution is claiming from the Wellcome Trust grant (to cover the OA publishing fee).  
   - These costs exclude any page or colour charges which the publisher may levy. The cost quoted, includes VAT where applicable. All costs have been converted into £ sterling.
   - As you look through the data you will see (for example) different prices cited for the same journal.  When these differences are small, it can be assumed that this reflects fluctuations in currency prices.  Where they are significant, then it should be assumed that other funders have contributed to the cost of that APC, and the figure cited is the Wellcome contribution to that APC.

## Data Preview

In [6]:
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


Additional Insights about the Dataset

In [7]:
print(df.info(), '\n')
print(df.dtypes, '\n')
print("\n",df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB
None 

PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object 


                PMID/PMCID Publisher Journal title  \
count                1928      2127          2126   
unique               1880       

## Data Cleaning

#### Rename Column Headers

In [8]:
#set new column names
cols = ["pmcid/pmid","publisher","journal_title","article_title","cost(£)"]

#Apply new column names to df
df.columns=cols

#check
df.head(2)

,pmcid/pmid,publisher,journal_title,article_title,cost(£)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04


#### Correct Column Types

Remove the £ symbol from each cell in the **cost(£)** column and change column data type to numeric.

In [9]:
#Replace £ with ''
df['cost(£)']=df['cost(£)'].str.replace('£','')

#there are $ throughout the column, removing
df['cost(£)']=df['cost(£)'].str.replace('$','')

##Convert cost(£) to numeric
df['cost(£)']=pd.to_numeric(df['cost(£)'])

#Check
df.dtypes

pmcid/pmid        object
publisher         object
journal_title     object
article_title     object
cost(£)          float64
dtype: object

#### Investigate Missing Data

Print number of missing values per column

In [10]:
print("# of missing values per column: \n", df.isnull().sum())

# of missing values per column: 
 pmcid/pmid       199
publisher          0
journal_title      1
article_title      0
cost(£)            0
dtype: int64


For the most part, we are missing about 200 ID's but we do have the majority of the rows and columns. **Let's first investigate that missing Journal title.**

In [11]:
#let's look into the null value for the journal title
df[df.journal_title.isnull()]

,pmcid/pmid,publisher,journal_title,article_title,cost(£)
986,NaN,MacMillan,NaN,Fungal Disease in Britain and the United State...,13200.0


At index 986, row is missing both a value in the pmcid/pmid and journal_title column. Checking for duplicates for the article_title column. Additionally, check for additional duplicate rows in the dataset and remove them.

In [12]:
#Check for any duplicated article titles, specifically for index 986
print(df[df.article_title.duplicated(keep=False)])

#Remove any duplicated rows
df.drop_duplicates(inplace=True)

     pmcid/pmid                  publisher journal_title  \
1490    Pending  Public Library of Science      PLoS One   
1496        NaN  Public Library of Science      PLoS One   

                                          article_title  cost(£)  
1490  Exclusive breastfeeding, diarrhoel morbidity a...   825.68  
1496  Exclusive breastfeeding, diarrhoel morbidity a...   825.68  


In [13]:
#insert unknown values for missing info @index 986 
df.loc[(df.journal_title.isnull()), 'pmcid/pmid']='Unknown'
df.loc[(df.journal_title.isnull()), 'journal_title']='Unknown'

#Check
df.iloc[986,:]

pmcid/pmid                                                 Unknown
publisher                                                MacMillan
journal_title                                              Unknown
article_title    Fungal Disease in Britain and the United State...
cost(£)                                                      13200
Name: 986, dtype: object

**Missing values in pmcid/pmid column**

Because we have no missing values in the columns other than pmcid/pmc, I will replace those null values with "UNKNOWN". Someone with domain knowledge may be able to reverse engineer the ID's using the publisher, journal_title, article_title, and cost(£) columns.

In [14]:
#Replace NA's with 'Unknown'
df['pmcid/pmid'].fillna('UNKNOWN', inplace=True)

# Apply RegEx to cleaning pmid/pmcid column

A couple questions about the ID's in this column before I make any changes:
* What are the differences between PMC/PubMed ID vs the PMC ID?
* What is the best way to track progress through cleaning this column?
    * Would it be best to define a format and count the number of cells that do not meet this format(format being length of the string)?

**What are the differences between the PMC ID and PubMedID?**

In [15]:
#Extract cells containing PMC
print("PMC ID's: \n", df['pmcid/pmid'][df['pmcid/pmid'].str.contains('PMC')].head(5))

#Extract cells containing PMCID
print("\nPMCID ID's: \n", df['pmcid/pmid'][df['pmcid/pmid'].str.contains('PMCID')].tail(5))

#Extract cells containing PMID
print("\nPMID ID's: \n", df['pmcid/pmid'][df['pmcid/pmid'].str.contains('PMID')].head(5))

PMC ID's: 
 1               PMC3679557
2    23043264  PMC3506128 
3      23438330 PMC3646402
4     23438216 PMC3601604 
5               PMC3579457
Name: pmcid/pmid, dtype: object

PMCID ID's: 
 2090           PMCID: PMC3779110
2101           PMCID: PMC3664106
2106           PMCID: PMC3600532
2111           PMCID: PMC3728731
2125    PMCID:\n    PMC3647051\n
Name: pmcid/pmid, dtype: object

PMID ID's: 
 19    PMID: 24015914 PMC3833349 
29                PMID: 20146481
31      PMID:23281892 PMC3579312
49    PMID: 23477577 PMC3727344 
59       PMID: 23409903 23409903
Name: pmcid/pmid, dtype: object


PMC ID's contain 7 digits while PMID's contain 8 digits. Additionally, some of these cells contain both kinds of ID's while others only contain one kind of ID.

**What is the best way to track progress through cleaning this column?**

My initial thoughts are that I would like all values in the pmcid/pmid column to have one of the following fomats:
* String for missing values: UNKNOWN 
* 10 character length string indicating PMC ID formatted like so: PMC#######
* 12 character length string indicating PMID formatted like so: PMID########
* 23 character length string indicating PMC ID and PMID in the same cell formatted either: PMC####### PMID######## | PMID######## PMC#######

In [16]:
#Count rows that meet condition.Subtract from total # of ID's to determine number of ID's that need cleaning 
def condition1(x):
    length=0
    if x == 'UNKNOWN':
        length +=1
    elif len(str(x)) == 10:
        length +=1
    elif len(str(x)) == 12:
        length +=1
    elif len(str(x)) == 23:
        length +=1
        return length
    else:
        length += 0
    return length

Easiest place to start would be to standardize this column as best we can
* Strip Whitespace
* Convert column to all uppercase
* Locate 7 digit strings and add PMC to front
* Locate 8 digit strings and add PMID to front
* Remove all instances of PMCID
* Replace - with UNKNOWN
* Replace \s+ with \s
* Strip Whitespace 

In [17]:
#Let's also remove \n and strip trailing and leading white spaces
df['pmcid/pmid']=df['pmcid/pmid'].str.strip()

#Convert column to all uppercase
df['pmcid/pmid']=df['pmcid/pmid'].str.upper()

#Locate 7 digit strings and add PMC to front
df['pmcid/pmid']=df['pmcid/pmid'].apply(lambda x: 'PMC' + x if any(pd.Series(x).str.contains(r'^[0-9]{7,7}$')) else x)

#Locate 8 digit strings and add PMID to front
df['pmcid/pmid']=df['pmcid/pmid'].apply(lambda x: 'PMID' + x if any(pd.Series(x).str.contains(r'^[0-9]{8,8}')) else x)

#Remove all instances of PMCID
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'PMCID:*',' ', regex=True)

#Replace - with UNKNOWN
df['pmcid/pmid']=df['pmcid/pmid'].apply(lambda x: 'UNKNOWN' if any(pd.Series(x).str.contains(r'^\s*\-\s*$')) else x)

#Replace \s+ with \s
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\s+', ' ', regex=True)

#Strip White space
df['pmcid/pmid']=df['pmcid/pmid'].str.strip()

#number of ID's to investigate
print("Number of ID's left to investigate: ", len(df)-sum(df['pmcid/pmid'].apply(condition1)))

Number of ID's left to investigate:  341


To Do
* Remove any numbers less than 7 integers in length
* Replace any strings without digits with UNKNOWN
* Remove :
* Fill blanks with UNKNOWN
* Strip white space

In [18]:
#Remove any numbers less than 7 integers in length
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'(^|\s)[0-9]{1,6}(\s|,|$)', ' ', regex=True, case=False)

#Replace any strings without an digits with UNKNOWN
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'^[\D]+$', 'UNKNOWN', regex=True, case=False)

#Remove :
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\s*\:', '', regex=True, case=False)

#Fill Blanks with UNKNOWN
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'^\s*$', 'UNKNOWN', regex=True)

#Strip White space
df['pmcid/pmid']=df['pmcid/pmid'].str.strip()

#number of ID's to investigate
print("Number of ID's left to investigate: ", len(df)-sum(df['pmcid/pmid'].apply(condition1)))

Number of ID's left to investigate:  209


TO DO
* Remove /, (), [],-
* Replace PMID\s with PMID
* Replace PMC\s with PMC
* Remove PMID Clones
* Add PMC in front of 7 digit numbers
* Fix the \s\s
* Strip Whitespace

In [19]:
#Remove /, (), [],-
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'[\/\-(),]', ' ', regex=True, case=False)

#Replace PMID\s with PMID
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'^PMID\s', 'PMID', regex=True, case=False)

#Remove PMID Clones
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\s[0-9]{8,8}', '', regex=True, case=False)

#Add PMC in front of 7 digit numbers
df['pmcid/pmid']=df['pmcid/pmid'].apply(lambda x: 'PMC' + x if any(pd.Series(x).str.contains(r'^[0-9]{7,7}$')) else x)


#Replace PMC\s with PMC
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'PMC\s', 'PMC', regex=True, case=False)

#fix \s\s
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\s\s+', ' ', regex=True, case=False)

#Strip White space
df['pmcid/pmid']=df['pmcid/pmid'].str.strip()

#number of ID's to investigate
print("Number of ID's left to investigate: ", len(df)-sum(df['pmcid/pmid'].apply(condition1)))

Number of ID's left to investigate:  14


TO DO
* Remove 6 digit PMC344470
* Remove PMID23495205685.88
* Correct MC3489778
* Remove the random jibberish

In [20]:
#Remove 6 digit PMC344470
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\sPMC344470', '', regex=True, case=False)

#Remove PMID23495205685.88
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'PMID23495205685.88\s', '', regex=True, case=False)


#Correct MC3489778
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'MC3489778', 'PMC3489778', regex=True, case=False)

#Remove the random jibberish

#add PMC in front of 7 digit number
df['pmcid/pmid']=df['pmcid/pmid'].apply(lambda x: 'PMC' + x if any(pd.Series(x).str.contains(r'(^[0-9]{7,7}|\s[0-9]{7,7}$)')) else x)

#Remove Bracket and text
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'\s(\[.*\])', '', regex=True, case=False)

#Clean-up
df['pmcid/pmid']=df['pmcid/pmid'].str.replace(r'Id is\s', '', regex=True, case=False)

#manually update remaining
df.iloc[367,0]='PMID23931634'
df.iloc[1036,0]= 'PMID23703681'
df.iloc[1052,0]= 'PMC3476715'
df.iloc[1071,0]= 'PMID23962810'
df.iloc[1231,0]= 'PMID24048963'
df.iloc[1716,0]= 'PMID23828613'
df.iloc[2126,0]= 'PMID23846567'

#Strip White space
df['pmcid/pmid']=df['pmcid/pmid'].str.strip()

#number of ID's to investigate
print("Number of ID's left to investigate: ", len(df)-sum(df['pmcid/pmid'].apply(condition1)))

Number of ID's left to investigate:  1


C:\Users\kishe\Miniconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [21]:
df['pmcid/pmid'].loc[condition].head(11)

NameError: name 'condition' is not defined

Perfect, let's move onto cleaning up the Publisher Column

### Cleaning Publisher Title column

**Metrics**
I think it would be wise to attempt to reduce the number of unique Journal Titles as much as possible. 

In [ ]:
#print('\nUnique Journal Titles are: \n', df.publisher.unique())
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

Let's start with the easy fixes:
* Turn text into all-caps
* Replace & with AND
* Remove INC, LTD,LLC \, -, (), and ,
* Remove any text relating to publishing(redundant)
* Strip White Space and \n

In [ ]:
#Turn text into all-caps
df['publisher']=df['publisher'].str.upper()

# Replace & with AND
df['publisher']=df['publisher'].str.replace(r'&', 'AND', regex=True)

#Remove INC, LTD,LLC, \, -, (), and ,
df['publisher']=df['publisher'].str.replace(r'[-(),\/]', ' ', regex=True)
df['publisher']=df['publisher'].str.replace(r'\bltd|inc|llc\.*\s*\b', '', case=False)

#Remove publishing @ end of strings
df['publisher']=df['publisher'].str.replace(r'publish(ing|ers|er)\.*$', '', regex=True, case=False)
df['publisher']=df['publisher'].str.replace(r'publication(s|\s*)$', ' ', regex=True, case=False)


#Strip White Space
df['publisher']=df['publisher'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

In [ ]:
# df.publisher.unique()

For the remaining cells to clean, I will approach by Journal Title to standardize titles and correct mispellings that are specific to each title.

To Do (Grouping Journals starting with A-C)
* Replace ACS to American Chemical Society
* AMERICAN PSYCHIATRIC ASSOCIATION
* Straighten out the spelling for 'AMERICAN SOCIETY FOR BIOCHEMISTRY AND MOLECULAR BIOLOGY'
* Straighten out the spelling for 'AMERICAN SOCIETY OF HEMATOLOGY'
* Replace ASM  with AMERICAN SOCIETY FOR MICROBIOLOGY
* BENTHAN SCIENCE should be BENTHAM SCIENCE
* BLACKWELL  WILEY should be WILEY BLACKWELL  
* CADMUS JOURNAL SERVICES
* CAMBRIDGE UNIVERSITY PRESS'
* COLD SPRING HARBOR LABORATORY PRESS

In [ ]:
#American Chemical Society
df['publisher']=df['publisher'].str.replace(r'ACS.*', 'AMERICAN CHEMICAL SOCIETY', case=False, regex=True)

#AMERICAN PSYCHIATRIC ASSOCIATION
df['publisher']=df['publisher'].str.replace(r'.*PSYCHIATRIC.*', 'AMERICAN PSYCHIATRIC ASSOCIATION',case=False, regex=True)

#Straighten out the spelling for 'AMERICAN SOCIETY FOR BIOCHEMISTRY AND MOLECULAR BIOLOGY'
df['publisher']=df['publisher'].str.replace(r'^American\sSoc(\s|iety).*', 
                                            'THE AMERICAN SOCIETY FOR BIOCHEMISTRY AND MOLECULAR BIOLOGY',case=False, regex=True)

#Straighten out the spelling for 'AMERICAN SOCIETY OF HEMATOLOGY'
df['publisher']=df['publisher'].str.replace(r'H(AE|E)MATOLOGY', 'HEMATOLOGY', case=False, regex=True)
                                            
#Replace ASM  with AMERICAN SOCIETY FOR MICROBIOLOGY
df['publisher']=df['publisher'].str.replace(r'^ASM.*', 'AMERICAN SOCIETY FOR MICROBIOLOGY', case=False, regex=True)

#BENTHAN SCIENCE should be BENTHAM SCIENCE
df['publisher']=df['publisher'].str.replace(r'BENTHAN', 'BENTHAM', case=False, regex=True)

#BLACKWELL  WILEY should be WILEY BLACKWELL 
df['publisher']=df['publisher'].str.replace(r'^blackwell\s+wiley','WILEY BLACKWELL', case=False, regex=True)

#Correct CADMUS JOURNAL SERVICES
df['publisher']=df['publisher'].str.replace(r'CA(DM|MD)US.*','CADMUS JOURNAL SERVICES', case=False, regex=True)

#CAMBRIDGE UNIVERSITY PRESS and CUP = CAMBRIDGE UNIVERSITY PRESS
df['publisher']=df['publisher'].str.replace(r'^(CUP|cambridge).*', 
                                            'CAMBRIDGE UNIVERSITY PRESS',case=False, regex=True)

#COLD SPRING HARBOR LABORATORY PRESS
df['publisher']=df['publisher'].str.replace(r'COLD.*', 
                                            'COLD SPRING HARBOR LABORATORY PRESS',case=False, regex=True)

#THE COMPANY OF BIOLOGISTS
df['publisher']=df['publisher'].str.replace(r'.*biol(ogi|gi)st[a-z]*$', 'THE COMPANY OF BIOLOGISTS',case=False, regex=True)
                                            
#Strip White Space
df['publisher']=df['publisher'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

In [ ]:
#df.publisher.unique()

Next Set of To Do for D-H
* Dartmouth Journal Services
* Elsevier
* THE ENDOCRINE SOCIETY
* EUROPEAN SOCIETY OF ENDOCRINOLOGY
* FASEB
* FRONTIERS MEDIA
* HINDAWI
* HUMANA PRESS  SPRINGER IMPRINT(spacing)


In [ ]:
#Dartmouth Journal Services
df['publisher']=df['publisher'].str.replace(r'da(rt|r)mouth.*',
                                            'DARTMOUTH JOURNAL SERVICES',case=False, regex=True)
#Elsevier
df['publisher']=df['publisher'].str.replace(r'^Elsev(eie|ie)r.*', 'ELSEVIER',case=False, regex=True)

#THE ENDOCRINE SOCIETY
df['publisher']=df['publisher'].str.replace(r'\w*\s*end(ro|o)crine soc(ie|e)ty', 'THE ENDOCRINE SOCIETY',case=False, regex=True)

#EUROPEAN SOCIETY OF ENDOCRINOLOGY
df['publisher']=df['publisher'].str.replace(r'ENDOCRINOLOG$', 'ENDOCRINOLOGY',case=False, regex=True)

#FASEB -> FEDERATION OF AMERICAN SOCIETIES FOR EXPERIMENTAL BIOLOGY
df['publisher']=df['publisher'].str.replace(r'^(Faseb|Federation.*)', 'FEDERATION OF AMERICAN SOCIETIES FOR EXPERIMENTAL BIOLOGY',case=False, regex=True)

#FRONTIERS MEDIA
df['publisher']=df['publisher'].str.replace(r'^FRONTIERS.*', 'FRONTIERS MEDIA',case=False, regex=True)

#HUMANA PRESS SPRINGER IMPRINT(spacing)
df['publisher']=df['publisher'].str.replace(r'^HUMANA.*', 'HUMANA PRESS SPRINGER IMPRINT',case=False, regex=True)

#Strip White Space
df['publisher']=df['publisher'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

In [ ]:
#df.publisher.unique()

TO DO I-P

* INFORMA HEALTHCARE
* (IUCr) INTERNATIONAL UNION OF CRYSTALLOGRAPHY 
* Journal of Medical Internet Research J MED INTERNET RESEARCH
* JOHN WILEY(check along bottom with wiley blackwell)
* THE JOURNAL OF VISUALIZED EXPERIMENTS
* LANDES BIOSCIENCE
* LWW; Lippincott Williams & Wilkins
* MARY ANN LIEBERT
* MIT PRESS
* MYJoVE Corporation
* NATIONAL ACADEMY OF SCIENCES
* NATURE
* OPEN ACCESS REG
* OXFORD UNIVERSITY PRESS(OUP)
* (PLOS) PUBLIC LIBRARY OF SCIENCE
* PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES (PNAS)
* SOCIETY FOR ENDOCRINOLOGY


In [ ]:
#INFORMA HEALTHCARE
df['publisher']=df['publisher'].str.replace(r'INFORMA.*','INFORMA HEALTHCARE',case=False, regex=True)

#(IUCr) INTERNATIONAL UNION OF CRYSTALLOGRAPHY 
df['publisher']=df['publisher'].str.replace(r'.*CRYSTALLOGRAPHY.*',
                                            'INTERNATIONAL UNION OF CRYSTALLOGRAPHY',case=False, regex=True)

#Journal of Medical Internet Research J MED INTERNET RESEARCH
df['publisher']=df['publisher'].str.replace(r'^j\s.*',
                                            'JOURNAL OF MEDICAL INTERNET RESEARCH',case=False, regex=True)

#INSTITUTE OF PHYSICS(IOP)
df['publisher']=df['publisher'].str.replace(r'^IOP$',
                                            'INSTITUTE OF PHYSICS',case=False, regex=True)

##JOHN WILEY, WILEY AND SON, WILEY BLACKWELL are all a part of parent company WILEY
df['publisher']=df['publisher'].str.replace(r'.*W(li|IL)EY.*',
                                            'WILEY',case=False, regex=True)

##THE JOURNAL OF VISUALIZED EXPERIMENTS(JOVE)
df['publisher']=df['publisher'].str.replace(r'^JOURNAL OF VISUALIZED.*|^JOVE',
                                            'THE JOURNAL OF VISUALIZED EXPERIMENTS',case=False, regex=True)

#LANDES BIOSCIENCE
df['publisher']=df['publisher'].str.replace(r'LANDES BIOSCIENCE.*',
                                          'LANDES BIOSCIENCE',case=False, regex=True)

#LWW; Lippincott Williams AND Wilkins
df['publisher']=df['publisher'].str.replace(r'LWW',
                                            'LIPPINCOTT WILLIAMS AND WILKINS',case=False, regex=True)

#MARY ANN LIEBERT
df['publisher']=df['publisher'].str.replace(r'MARY ANN LIEBERT.*',
                                            'MARY ANN LIEBERT',case=False, regex=True)

#MIT PRESS
df['publisher']=df['publisher'].str.replace(r'^MIT\s.*',
                                            'MIT PRESS',case=False, regex=True)

#MYJoVE Corporation
df['publisher']=df['publisher'].str.replace(r'^MY(\s|\s*)JOVE.*',
                                            'MYJOVE CORPORATION',case=False, regex=True)

#NATIONAL ACADEMY OF SCIENCES
df['publisher']=df['publisher'].str.replace(r'^NATIONAl.*',
                                            'NATIONAL ACADEMY OF SCIENCES',case=False, regex=True)

#NATURE
df['publisher']=df['publisher'].str.replace(r'^NATURE.*',
                                            'NATURE PUBLISHING GROUP',case=False, regex=True)

#OXFORD UNIVERSITY PRESS(OUP)
df['publisher']=df['publisher'].str.replace(r'^OUP|oxford.*',
                                            'OXFORD UNIVERSITY PRESS',case=False, regex=True)

#(PLOS) PUBLIC LIBRARY OF SCIENCE
df['publisher']=df['publisher'].str.replace(r'^plos.*',
                                            'PUBLIC LIBRARY OF SCIENCE',case=False, regex=True)

#PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES (PNAS)
df['publisher']=df['publisher'].str.replace(r'^(PNAS.*|Proceedings).*',
                                            'PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES',case=False, regex=True)

#SOCIETY FOR ENDOCRINOLOGY
df['publisher']=df['publisher'].str.replace(r'^PUBLISHER',
                                            '',case=False, regex=True)

#Strip White Space
df['publisher']=df['publisher'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

In [ ]:
#df.publisher.unique()

Q-END TO DO LIST
* PUBMED

* THE ROYAL SOCIETY OF CHEMISTRY(RSC)
* SAGE PUBLICATIONS
* SOCIETY FOR(OF) GENERAL MICROBIOLOGY
* SOCIETY FOR(OF) NEUROSCIENCE
* SPRINGER PUBLISHING
* TAYLOR AND FRANCIS
* THE SHERIDAN PRESS
* WOLTERS KLUWER

In [ ]:
#PUBMED
df['publisher']=df['publisher'].str.replace(r'^pubmed.*',
                                            'PUBMED',case=False, regex=True)

# THE ROYAL SOCIETY OF CHEMISTRY(RSC)
df['publisher']=df['publisher'].str.replace(r'^RSC|.*Royal\sSociety.*',
                                            'THE ROYAL SOCIETY OF CHEMISTRY',case=False, regex=True)
# SAGE PUBLICATIONS
df['publisher']=df['publisher'].str.replace(r'^Sage.*',
                                            'SAGE PUBLICATIONS',case=False, regex=True)
# SOCIETY FOR(OF) GENERAL MICROBIOLOGY
df['publisher']=df['publisher'].str.replace(r'.*gener(a|ma)l.*',
                                            'SOCIETY FOR GENERAL MICROBIOLOGY',case=False, regex=True)
# SOCIETY FOR(OF) NEUROSCIENCE
df['publisher']=df['publisher'].str.replace(r'.*NEUROSCIENCE(s|.*)$',
                                            'SOCIETY FOR NEUROSCIENCE',case=False, regex=True)
#SPRINGER PUBLISHING
df['publisher']=df['publisher'].str.replace(r'^springer.*',
                                            'SPRINGER PUBLISHING',case=False, regex=True)
#TAYLOR AND FRANCIS
df['publisher']=df['publisher'].str.replace(r'^taylor.*|^TANDF',
                                            'TAYLOR AND FRANCIS',case=False, regex=True)
# THE SHERIDAN PRESS
df['publisher']=df['publisher'].str.replace(r'.*SHERIDAN.*',
                                            'THE SHERIDAN PRESS',case=False, regex=True)
# WOLTERS KLUWER
df['publisher']=df['publisher'].str.replace(r'.*KLUWER.*',
                                            'WOLTERS KLUWER',case=False, regex=True)


#SOCIETY FOR LEUKOCYTE BIOLOGY
df['publisher']=df['publisher'].str.replace(r'.*LEUKOCYTE.*',
                                            'SOCIETY FOR LEUKOCYTE BIOLOGY',case=False, regex=True)

#ACTA DERMATO VENEREOLOGICA
df['publisher']=df['publisher'].str.replace(r'.*VENEREOLOGICA.*',
                                            'ACTA DERMATO VENEREOLOGICA',case=False, regex=True)
#Strip White Space
df['publisher']=df['publisher'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.publisher.nunique())

In [ ]:
#df.publisher.unique()

This looks great so far. Let's finish up our cleaning effots on the Journal Title column and we wil move onto answering some of the stats questions about the data set.

### Cleaning Journal Title column

**Metrics**
I think it would be wise to attempt to reduct the number of unique Journal Titles as best as possible. 

In [ ]:
print('The number of unique Journal Titles in dataset: \n',df.journal_title.nunique())
#print('\nUnique Journal Titles are: \n', df.journal_title.unique())

We will have our work cut out for us here. Let's start with the easy stuff
* Capitalize all Characters
* Replace J with JOURNAL OF
* Replace & with AND
* Replace med with MEDICINAL
* Repalce chem with CHEMISTRY
* BIOL with BIOLOGICAL
* Org with ORGANIC
* Pharm with PHARMACY
* mol with MOLECULAR 
* Strip White Space

In [ ]:
#Capitalize all Characters
df['journal_title']=df['journal_title'].str.upper()

#Replace ^J's with JOURNAL OF
df['journal_title']=df['journal_title'].str.replace(r'J\s',
                                                    'JOURNAL OF ', case=False,regex=True)
#Replace & with AND
df['journal_title']=df['journal_title'].str.replace(r'&',
                                                    'AND', case=False,regex=True)
#Replace med with MEDICINAL 
df['journal_title']=df['journal_title'].str.replace(r'med\s',
                                                    'MEDICINAL ', case=False,regex=True)
#Repalce chem with CHEMISTRY
df['journal_title']=df['journal_title'].str.replace(r'chem($|\s|\.)',
                                                    'CHEMISTRY', case=False,regex=True)

#BIOL with BIOLOGICAL
df['journal_title']=df['journal_title'].str.replace(r'biol($|\s|\.)',
                                                    'BIOLOGICAL ', case=False,regex=True)

#Org with ORGANIC
df['journal_title']=df['journal_title'].str.replace(r'org\s',
                                                    'ORGANIC ', case=False,regex=True)

#Pharm with PHARMACY
df['journal_title']=df['journal_title'].str.replace(r'PHARM($|\s)',
                                                    'PHARMACY', case=False,regex=True)
#mol with MOLECULAR 
df['journal_title']=df['journal_title'].str.replace(r'mol\s',
                                                    'MOLECULAR ', case=False,regex=True)

# #Strip White Space
df['journal_title']=df['journal_title'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.journal_title.nunique())

In [ ]:
#print('\nUnique Journal Titles are: \n', df.journal_title.unique())

TO DO
* Replace THE JOURNAL with JOURNAL
* Replace \s\s with \s
* Replace clin with CLINICAL
* Replace hum with HUMAN
* AGFENTS with AGENTS
* Sci(.) with SCIENCE
* Remove .
* infec with INFECTION
* Rogue (
* bio with BIOLOGY
* IJTLD with International Journal of Tuberculosis and Lung Disease
* strip white space

In [ ]:
#Replace THE JOURNAL with JOURNAL
df['journal_title']=df['journal_title'].str.replace(r'THE JOURNAL',
                                                    'JOURNAL', case=False,regex=True)
#Replace \s\s with \s
df['journal_title']=df['journal_title'].str.replace(r'\s\s',
                                                    ' ', case=False,regex=True)
#Replace clin with CLINICAL
df['journal_title']=df['journal_title'].str.replace(r'clin(\s|\.|$)',
                                                    'CLINICAL', case=False,regex=True)
#Replace hum with HUMAN
df['journal_title']=df['journal_title'].str.replace(r'hum(\s|\.|$)',
                                                    'HUMAN', case=False,regex=True)
#AGFENTS with AGENTS
df['journal_title']=df['journal_title'].str.replace(r'ag(e|fe)nt',
                                                    'AGENT', case=False,regex=True)
#Sci(.) with SCIENCE
df['journal_title']=df['journal_title'].str.replace(r'sci\.',
                                                    'SCIENCE', case=False,regex=True)
#Remove .
df['journal_title']=df['journal_title'].str.replace(r'\.',
                                                    '', case=False,regex=True)
#infec with INFECTION
df['journal_title']=df['journal_title'].str.replace(r'infec',
                                                    'INFECTION', case=False,regex=True)

#Remove Rogue (
df['journal_title']=df['journal_title'].str.replace(r'\($',
                                                    '', case=False,regex=True)
#Bio with BIOLOGY
df['journal_title']=df['journal_title'].str.replace(r'bio\s',
                                                    'BIOLOGY', case=False,regex=True)

#IJTLD with INTERNATIONAL JOURNAL OF TUBERCULOSIS AND LUNG DISEASE
df['journal_title']=df['journal_title'].str.replace(r'IJTLD',
                                                    'INTERNATIONAL JOURNAL OF TUBERCULOSIS AND LUNG DISEASE', case=False,regex=True)
#Strip White Space
df['journal_title']=df['journal_title'].str.strip()

#Number of Unique Journal Titles remaining
print('The number of unique Publisher Titles in dataset: \n',df.journal_title.nunique())

There are still some issues remaining but I have cleaned enough where we can do a statistical analysis on the data. I am not too concerned about the remaining errors with the Journal titles namely because the errors are minor and the PMID/PMC as well as the publisher/article info should be enough to pull up the relevant journal titles as needed.

## STATISTICAL ANALYSIS

**Determine the five most common journals and the total articles for each.**

In [ ]:
#Aggreagate article counts, sorted largest to smallest, by journal
count= df.groupby('journal_title')['article_title'].count().sort_values(ascending=False).reset_index()

#Rename stats column headers for easy interpretation
cols= ['Journal', 'Article_Count']
count.columns=cols

#Display five most common journals and the total articles for each
count.head(5)

**Calculate the mean, median, and standard deviation of the open-access cost per article for each journal.**

In [ ]:
#Replace 999999.0 cost with nan
df['cost(£)']=df['cost(£)'].replace(999999.0 , np.nan)

#Aggregate stats for each journal title
stats=df.groupby('journal_title')['cost(£)'].agg({'AVG COST':'mean', 'MEDIAN COST':'median','COST STD':'std'})

#Display stats
stats